# SastaTicket Datathon Flight Price Prediction and Analysis

In [3]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import metrics
import warnings
import pickle
import category_encoders as ce
warnings.filterwarnings('ignore')

In [4]:
# Set maximum number of columns
pd.set_option('display.max_columns', None)

In [5]:
# Set Number of Samples from the file to be trained ( Its for Normal Computer If you have a good high level server then full data can be trained)
n_samples = 21776590

# Importing dataset¶
1.Since data is in form of csv file we have to use pandas read_csv to load the data.

2.After loading it is important to check null values in a column or a row

3.If it is present then following can be done,

Filling NaN values with mean, median and mode using fillna() method
If Less missing values, we can drop it as well

In [6]:
# loading x_test.csv file
df_X_test = pd.read_csv("X_test.csv")

In [7]:
#information about the data
df_X_test.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4532489 entries, 0 to 4532488
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   f1      object 
 1   f2      object 
 2   f3      object 
 3   f4      object 
 4   f5      object 
 5   f6      object 
 6   f7      bool   
 7   f8      float64
 8   f9      int64  
 9   f10     object 
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 315.5+ MB


In [8]:
#describe about the data
df_X_test.describe()

,f8,f9
count,4.532489e+06,4.532489e+06
mean,2.201668e+01,9.891111e-01
std,7.738736e+00,5.539422e-01
min,0.000000e+00,0.000000e+00
25%,2.000000e+01,1.000000e+00
50%,2.000000e+01,1.000000e+00
75%,2.000000e+01,1.000000e+00
max,4.000000e+01,2.000000e+00


In [9]:
#checking total number of rows and columns
print(df_X_test.shape) 
#Initial structure of the data
df_X_test.head()

(4532489, 10)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,2021-09-01 08:30:19.320999+00:00,x,y,2021-09-08 11:00:00+00:00,2021-09-08 12:55:00+00:00,beta,True,40.0,0,b-69
1,2021-09-01 08:30:19.320999+00:00,x,y,2021-09-08 14:00:00+00:00,2021-09-08 15:55:00+00:00,beta,True,40.0,0,b-9
2,2021-09-01 08:30:19.320999+00:00,x,y,2021-09-08 05:00:00+00:00,2021-09-08 06:55:00+00:00,beta,True,20.0,0,b-1
3,2021-09-01 08:30:20.396761+00:00,x,y,2021-09-08 05:30:00+00:00,2021-09-08 07:30:00+00:00,gamma,True,20.0,2,c-2
4,2021-09-01 08:30:20.396761+00:00,x,y,2021-09-08 05:30:00+00:00,2021-09-08 07:30:00+00:00,gamma,True,20.0,1,c-2


In [10]:
#finding the null values
df_X_test.isnull().sum()

f1     0
f2     0
f3     0
f4     0
f5     0
f6     0
f7     0
f8     0
f9     0
f10    0
dtype: int64

In [11]:
# loading y_train.csv file
df_y_train = pd.read_csv("y_train.csv")

In [12]:
#Initial structure of the data
df_y_train.head()

,Unnamed: 0,target
0,0,7400.0
1,1,8650.0
2,2,9150.0
3,3,10400.0
4,4,8697.0


In [13]:
#Remove unwanted column
df_y_train.drop(['Unnamed: 0'],axis=1, inplace=True)

In [14]:
#information about the data
df_y_train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21776590 entries, 0 to 21776589
Data columns (total 1 columns):
 #   Column  Dtype  
---  ------  -----  
 0   target  float64
dtypes: float64(1)
memory usage: 166.1 MB


In [15]:
#describe about the data
df_y_train.describe()

,target
count,2.177659e+07
mean,1.013705e+04
std,3.447472e+03
min,0.000000e+00
25%,7.796000e+03
50%,9.380000e+03
75%,1.124500e+04
max,3.500000e+04


In [16]:
#checking total number of rows and columns
print(df_y_train.shape) 

(21776590, 1)


In [17]:
#Initial structure of the data
df_y_train.head()

,target
0,7400.0
1,8650.0
2,9150.0
3,10400.0
4,8697.0


In [18]:
#finding the null values
df_y_train.isnull().sum()

target    0
dtype: int64

In [19]:
#Read 
df_X_train = pd.read_csv("X_train.csv")

In [20]:
#Initial structure of the data
df_X_train.head(10)

,Unnamed: 0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,0,2020-12-31 09:46:17.463002+00:00,x,y,2021-01-10 05:00:00+00:00,2021-01-10 07:00:00+00:00,gamma,True,0.0,0,c-2
1,1,2020-12-31 09:46:17.463002+00:00,x,y,2021-01-10 05:00:00+00:00,2021-01-10 07:00:00+00:00,gamma,True,32.0,1,c-2
2,2,2020-12-31 09:46:17.463002+00:00,x,y,2021-01-10 11:00:00+00:00,2021-01-10 13:00:00+00:00,gamma,True,32.0,1,c-4
3,3,2020-12-31 09:46:17.463002+00:00,x,y,2021-01-10 11:00:00+00:00,2021-01-10 13:00:00+00:00,gamma,True,32.0,2,c-4
4,4,2020-12-31 09:46:18.191119+00:00,x,y,2021-01-25 11:00:00+00:00,2021-01-25 12:55:00+00:00,beta,False,20.0,0,b-69
5,5,2020-12-31 09:46:18.191119+00:00,x,y,2021-01-25 05:00:00+00:00,2021-01-25 06:55:00+00:00,beta,False,20.0,0,b-1
6,6,2020-12-31 09:46:18.191119+00:00,x,y,2021-01-25 14:00:00+00:00,2021-01-25 15:55:00+00:00,beta,False,20.0,0,b-9
7,7,2020-12-31 09:46:18.191119+00:00,x,y,2021-01-25 16:00:00+00:00,2021-01-25 17:55:00+00:00,beta,False,20.0,0,b-19
8,8,2020-12-31 09:46:19.878719+00:00,x,y,2021-01-25 11:00:00+00:00,2021-01-25 13:00:00+00:00,gamma,True,32.0,2,c-4
9,9,2020-12-31 09:46:19.878719+00:00,x,y,2021-01-25 11:00:00+00:00,2021-01-25 13:00:00+00:00,gamma,True,32.0,1,c-4


In [21]:
#information about the data
df_X_train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21776590 entries, 0 to 21776589
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   f1          object 
 2   f2          object 
 3   f3          object 
 4   f4          object 
 5   f5          object 
 6   f6          object 
 7   f7          bool   
 8   f8          float64
 9   f9          int64  
 10  f10         object 
dtypes: bool(1), float64(1), int64(2), object(7)
memory usage: 1.6+ GB


In [22]:
#describe about the data
df_X_train.describe()

,Unnamed: 0,f8,f9
count,2.177659e+07,2.177659e+07,2.177659e+07
mean,1.088829e+07,2.258956e+01,9.491727e-01
std,6.286360e+06,8.885366e+00,6.077294e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.444147e+06,2.000000e+01,1.000000e+00
50%,1.088829e+07,2.000000e+01,1.000000e+00
75%,1.633244e+07,3.200000e+01,1.000000e+00
max,2.177659e+07,4.500000e+01,2.000000e+00


In [23]:
#finding the null values
df_X_train.isnull().sum()

Unnamed: 0    0
f1            0
f2            0
f3            0
f4            0
f5            0
f6            0
f7            0
f8            0
f9            0
f10           0
dtype: int64

In [24]:
# Checking row of x and y train data set
df_X_train.shape[0] == df_y_train.shape[0]

True

In [25]:
# Logical names to the columns 
test_columns = ["Ticket Purchase Date Time","Origin","Destination","Departure Date Time",
                    "Arrival Date Time","Airline","Refundable Ticket","Baggage Weight","Baggage Pieces",
                    "Flight Number"]
def assignColumns(df):   

     # Dropif we have extra unused column
    if 'Unnamed: 0' in df:
        df = df.drop(['Unnamed: 0'],axis = 1)
    
    # Giving logical names to the column for better understanding
    df.columns = test_columns
    
    return df;


# Data Cleaning & Pre Processing



In [26]:
# defining date time formatter as per data 
dateTimeFormatter = '%Y-%m-%d %H:%M:%S.%f'


In [27]:

def prepareData(df):
    pd.options.mode.chained_assignment = None 
    
    df = assignColumns(df);
    
    # The datatypes of "Ticket Purchase Date Time","Departure Date Time" and "Arrival Date Time" is object.So,firstly we convert it into date and time for proper predicion.
    # dt.day method will extract only day of that date
    # dt.month method will extract only month of that date
    # dt.year method will extract only year of that date
    # dt.hour method will extract only hour of that date
    # dt.minute method will extract only minute of that date    
    
    df[[test_columns[0],test_columns[3],test_columns[4]]] = df[[test_columns[0],test_columns[3],test_columns[4]]].apply(pd.to_datetime, format=dateTimeFormatter)
    #seperating 
    df["Ticket Purchase Day"] = pd.to_datetime(df[test_columns[0]],format=dateTimeFormatter).dt.day
    df["Ticket Purchase Month"] = pd.to_datetime(df[test_columns[0]],format=dateTimeFormatter).dt.month
    df["Ticket Purchase Year"] = pd.to_datetime(df[test_columns[0]],format=dateTimeFormatter).dt.year
    df["Ticket Purchase Hour"] = pd.to_datetime(df[test_columns[0]],format=dateTimeFormatter).dt.hour
    df["Ticket Purchase_Minute"] = pd.to_datetime(df[test_columns[0]],format=dateTimeFormatter).dt.minute

    df["Departure Day"] = pd.to_datetime(df[test_columns[3]],format=dateTimeFormatter).dt.day
    df["Departure Month"] = pd.to_datetime(df[test_columns[3]],format=dateTimeFormatter).dt.month
    df["Departure Year"] = pd.to_datetime(df[test_columns[3]],format=dateTimeFormatter).dt.year
    df["Departure Hour"] = pd.to_datetime(df[test_columns[3]],format=dateTimeFormatter).dt.hour
    df["Departure Minute"] = pd.to_datetime(df[test_columns[3]],format=dateTimeFormatter).dt.minute
    
    df["Arrival Day"] = pd.to_datetime(df[test_columns[4]],format=dateTimeFormatter).dt.day
    df["Arrival Month"] = pd.to_datetime(df[test_columns[4]],format=dateTimeFormatter).dt.month
    df["Arrival Year"] = pd.to_datetime(df[test_columns[4]],format=dateTimeFormatter).dt.year
    df["Arrival Hour"] = pd.to_datetime(df[test_columns[4]],format=dateTimeFormatter).dt.hour
    df["Arrival Minute"] = pd.to_datetime(df[test_columns[4]],format=dateTimeFormatter).dt.minute

    # Calculating flight duration  by DepartHour and Arrival hours and minutes
    DepartHours = df["Departure Hour"] + (df["Departure Minute"]/60)
    ArrivalHours = df["Arrival Hour"] + (df["Arrival Minute"]/60)
    
    # Adding flight duration column in dataframe
    #df["FlightDuration"] = ArrivalHours - DepartHours
    
    # From "Ticket Purchase Date Time","Departure Date Time" and "Arrival Date Time" features, we extract day,month, year, hour and minute and stored in new columns and drop these columns¶
    df.drop([test_columns[0],test_columns[1],test_columns[2],test_columns[3], test_columns[4]], axis = 1, inplace = True)
    
    # As "Flight Number" is Nominal Categorical data we will perform OneHotEncoding
    #FlightNumber = pd.get_dummies(df[test_columns[9]], drop_first= True) 
    
    # Perform label encoding on "Refundable Ticket", "Airline" and  "Baggage Pieces" data
    df.replace({True: 1, False: 0}, inplace = True)
    #df.replace({'alpha': 0, 'gamma': 1, "beta":2,"omega":3}, inplace = True)
    df.replace({0: 0, 15:1, 20:2, 32:3, 35:4, 40:5, 45:6}, inplace = True)

    # Concatenate OneHotEncoding data
    #df = pd.concat([df, FlightNumber], axis = 1)
    
    # Drop "Flight Number" as new OneHotEncoding column is already added 
    #df.drop([test_columns[9]], axis = 1, inplace = True)

    # return clean dataframe
    return df


In [28]:
df_X_test = prepareData(df_X_test)

In [29]:
df_X_train = prepareData(df_X_train)

In [30]:
col = df_X_train.columns

# Count of flights with different Airlines

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Count of flights with different Airlines')
ax=sns.countplot(x = 'Airline', data = df_X_train)
plt.xlabel('Airline')
plt.ylabel('Count of flights')
plt.xticks(rotation = 90)
for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x()+0.25, p.get_height()+1), va='bottom',
                    color= 'black')

# Group by Airline

In [ ]:
df_X_train.groupby('Airline').sum()


# Airline vs Price

In [14]:
df_X_train['Price'] = df_y_train['target']

In [ ]:
fig = px.histogram(df_X_train, 
                   x='Price', 
                   marginal='box', 
                   color='Airline', 
                   color_discrete_sequence=['green', 'grey','red','orange','blue','blueviolet', 'brown', 'burlywood', 'chocolate', 'darkblue','olive','pink'], 
                   title='Airline vs Price')
fig.update_layout(bargap=0.1)
fig.show()

# Flight Number vs Price

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='Flight Number',y='Price',data=df_X_train.sort_values('Price',ascending=False))

# Airline, Flight Number and Price visualization with 3D view¶


In [ ]:
fig = px.scatter_3d(df_X_train, x='Airline', y='Flight Number', z='Price')
fig.update_traces(marker_size=3, marker_opacity=0.5)
fig.show()

# Finding corelation 

In [ ]:
plt.figure(figsize=(18,18))
sns.heatmap(df_X_test.corr(),annot=True,cmap='RdYlGn')
plt.show()

# FlightDuration vs Price visualization with scatter plot¶


In [ ]:
fig = px.scatter(df_X_train, 
                 x='FlightDuration', 
                 y='Price', 
                 #color='Source', 
                 opacity=0.8, 
                 hover_data=['Airline'], 
                 title='Flight Duration vs. Price')
fig.update_traces(marker_size=5)
fig.show()

# Frequency of Flight based on Arrival and Departure  day and month

In [ ]:
fig = plt.figure(figsize=(15,25))
plt.subplot(221) 
plt.hist(df_X_train['Arrival Day'] , bins=12)
plt.xlabel('Departure Day',fontsize = 15 , fontweight='bold' , fontstyle='italic')
plt.ylabel('Frequency',fontsize = 15 , fontweight='bold' , fontstyle='italic')

plt.subplot(222) 
plt.hist(df_X_train['Arrival Month'] , bins=12)
plt.xlabel('Arrival Month',fontsize = 15 , fontweight='bold' , fontstyle='italic')
plt.ylabel('Frequency',fontsize = 15 , fontweight='bold' , fontstyle='italic')

plt.subplot(223) 
plt.hist(df_X_train['Departure Day'] , bins=12)
plt.xlabel('Departure Day',fontsize = 15 , fontweight='bold' , fontstyle='italic')
plt.ylabel('Frequency',fontsize = 15 , fontweight='bold' , fontstyle='italic')


plt.subplot(224) 
plt.hist(df_X_train['Departure Month'] , bins=12)
plt.xlabel('Arrival Month',fontsize = 15 , fontweight='bold' , fontstyle='italic')
plt.ylabel('Frequency',fontsize = 15 , fontweight='bold' , fontstyle='italic')


plt.show()

Our observations:

- Most flights are in the month of Janruary
- Most flights are in 1st week of the all months.

# Day Expand and Cost visualization with Animation¶


In [ ]:
px.scatter(data_frame=df_X_train,
                x= 'Price',
                y='Departure Day',
                size='Departure Month',
                color='FlightDuration',
                title= 'Day Expand and Cost',
                labels= {'Price': 'Cost','Departure_Day': 'Day Expand'},
                log_x= True,
                range_y= [0,400],
                hover_name= 'Airline',
                animation_frame='Price',
                height= 400,
                size_max=40)

# Feature Selection / Training & Testing Model with Parameter Hypertuning


In [ ]:
df_X_train.drop(['Price'],axis=1, inplace=True)

In [31]:
#encoding
encoder = ce.OrdinalEncoder(cols=col)

In [32]:
# Encoding train & test data samples
x_train = encoder.fit_transform(df_X_train)
x_test = encoder.transform(df_X_test)

In [ ]:
# We have use a vast number of Models, but if the data is heavy then its recommend to use the models depending on your system.
algorithms =  {
    
    'RandomForestRegressor' : {
        'model' : RandomForestRegressor(),
        'param' : {
            'n_estimators' : [300, 500, 700, 1000, 2100],
            'max_depth' : [3, 5, 7, 9, 11, 13, 15],
            'max_features' : ["auto", "sqrt", "log2"],
            'min_samples_split' : [2, 4, 6, 8]
        }
    }

}

In [ ]:
score = []
for name, mp in algorithms.items() :
    rs = RandomizedSearchCV(estimator = mp['model'], param_distributions = mp['param'], cv = 5, n_jobs=-1, verbose=2)
    rs.fit(x_train, df_y_train)
    prediction = rs.predict(x_test)
    score.append({
        'ModelName': name,
        'Model': rs,
        'Score' : rs.best_score_,
        'Params' : rs.best_params_,
        'Predictions': prediction
    }) 

In [ ]:
ModelScore = pd.DataFrame (score, columns= ['Model', 'Score','Params','Predictions'])

In [ ]:
ModelScore

# Save Model for future use

In [ ]:
# save the model to disk
filename = 'finalized_models.sav'
pickle.dump(ModelScore, open(filename, 'wb'))



In [ ]:

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

In [32]:
x_train.shape,df_y_train.shape

((21776590, 21), (21776590, 1))

In [ ]:
# Random Forest Regressor

rfr_model = RandomForestRegressor(n_estimators=1000, n_jobs = -1)

rfr_model.fit(x_train,df_y_train)

In [81]:
x_train.shape,df_y_train.shape

((21776590, 21), (21776590, 1))

In [ ]:
# save the model to disk
filename = 'rfr_encod_model.sav'
pickle.dump(xg_reg, open(filename, 'wb'))

In [ ]:
#XGBoost hyper-parameter tuning
param_tuning = {'colsample_bytree': 0.7, 
 'learning_rate': 0.01, 
 'max_depth': 10, 
 'min_child_weight': 5,
 'n_estimators': 500, 
 'subsample': 0.5}





xgb_model = XGBRegressor()

gsearch = GridSearchCV(estimator = xgb_model,
                       param_grid = param_tuning,                        
                       #scoring = 'neg_mean_absolute_error', #MAE
                       #scoring = 'neg_mean_squared_error',  #MSE
                       cv = 2,
                       n_jobs = -1,
                       verbose = 1)

gsearch.fit(x_train,df_y_train)



In [50]:
from xgboost import XGBRegressor

#xg_reg = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
# Fitting the model
#xg_reg.fit(x_train,df_y_train,verbose=False)
xg_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05,n_jobs = -1)
xg_reg.fit(x_train,df_y_train, verbose=False)
# my_model.fit(train_X, train_y, early_stopping_rounds=5, 
#              eval_set=[(test_X, test_y)], verbose=False)


[12:06:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [51]:
# save the model to disk
filename = 'xg_reg.sav'
pickle.dump(xg_reg, open(filename, 'wb'))


In [ ]:
# y_pred_xgb = xgb_model.predict(X_val)

# mae_xgb = mean_absolute_error(y_val, y_pred_xgb)

# print("MAE: ", mae_xgb)

In [65]:
predications = xg_reg.predict(x_train.head())

In [61]:
predications = np.round(predications,1)

In [62]:
pd.DataFrame(predications).to_csv("output.csv",index=False,header=False)

In [ ]:
def generateOutputCSV(model_name, x_test, fileName):
    predications = model_name.predict(x_test)
    predications = np.round(predications,1)
    pd.DataFrame(predications).to_csv(fileName,index=False,header=False)
    

In [70]:
import math
error_xgb = math.sqrt(mean_squared_error(df_y_train, predications))
r2Score = metrics.r2_score(df_y_train,predications)
error_xgb,r2Score

(1597.7976358744781, 0.7851961120615718)

In [ ]:
# Xbg Regressor
# 78% r2 score


In [69]:
x_train.head()

,Airline,Refundable Ticket,Baggage Weight,Baggage Pieces,Flight Number,Ticket Purchase Day,Ticket Purchase Month,Ticket Purchase Year,Ticket Purchase Hour,Ticket Purchase_Minute,Departure Day,Departure Month,Departure Year,Departure Hour,Departure Minute,Arrival Day,Arrival Month,Arrival Year,Arrival Hour,Arrival Minute,FlightDuration
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,2,2,2,1,1,1,1,1,1,1,1,2,1,1,1,1,2,1,1
3,1,1,2,3,2,1,1,1,1,1,1,1,1,2,1,1,1,1,2,1,1
4,2,2,3,1,3,1,1,1,1,1,2,1,1,2,1,2,1,1,3,2,2


In [78]:

# load the model from disk
rfr_model = pickle.load(open('rfr_models-org.sav', 'rb'))


In [79]:
rfr_predictions = rfr_model.predict(x_train)

In [80]:
import math
error_xgb = math.sqrt(mean_squared_error(df_y_train, rfr_predictions))
r2Score = metrics.r2_score(df_y_train,rfr_predictions)
error_xgb,r2Score

(4971.536356115874, -1.0796000955903997)

In [ ]:
#Xgboost without flight duration

In [34]:
xgb_model_wo_duration = pickle.load(open('finalized_XGBModelwithoutdurationwithparam.sav', 'rb'))


In [35]:
score = xgb_model_wo_duration.score(x_train,df_y_train)
score

0.7925824450873259

In [ ]:
xgb_model_wo_duration_prediction = xgb_model_wo_duration.predict(x_train)

In [ ]:
import math
error_xgb = math.sqrt(mean_squared_error(df_y_train, predications))
r2Score = metrics.r2_score(df_y_train,predications)
error_xgb,r2Score

In [37]:
generateOutputCSV(xgb_model_wo_duration, x_test, "xbg_boost_wo_duration.csv"):

SyntaxError: invalid syntax (<ipython-input-37-f72d003a2f6f>, line 1)

In [38]:
xgb_predications = xgb_model_wo_duration.predict(x_test)
predications = np.round(xgb_predications,1)
pd.DataFrame(predications).to_csv("xbg_boost_wo_duration.csv",index=False,header=False)